In [107]:
import pandas as pd
import numpy as np
import math as math
from scipy.optimize import linprog

In [108]:
data = pd.read_csv('data/Rival Matrix - Ultimate and Winrate Optimization.csv')
data

,Name,Type,Win Rate,Damage Dealt Per Min,Healing Dealt Per Min,Damage Per 15 min Game,Healing Per 15 Min Game,Ult Cost,Time to Charge Ult (min),Ult Duration (s),Ult Duration (min),ults per game
0,Mantis,Strategists,54.31,840,1486,12600.0,22290.0,3700,3.167808,8.0,0.133333,4.543883
1,Adam Warlock,Strategists,53.61,972,1398,14580.0,20970.0,5000,4.201681,0.0,0.000000,3.570000
2,Groot,Vanguards,53.45,1755,0,26325.0,0.0,2800,1.590909,3.5,0.058333,9.095085
3,Star-Lord,Duelists,52.93,1782,0,26730.0,0.0,3100,1.734751,8.0,0.133333,8.029616
4,Loki,Strategists,52.00,731,2219,10965.0,33285.0,4000,2.702703,0.0,0.000000,5.550000
5,Magik,Duelists,51.90,1477,0,22155.0,0.0,3400,2.294197,0.0,0.000000,6.538235
6,Jeff,Strategists,51.79,776,1296,11640.0,19440.0,4000,3.842459,0.0,0.000000,3.903750
7,Captain America,Vanguards,51.73,1139,7,17085.0,105.0,2800,4.844291,10.0,0.166667,2.993440
8,Storm,Duelists,51.72,1956,0,29340.0,0.0,3100,1.580826,5.0,0.083333,9.013559
9,Hulk,Vanguards,50.93,1187,0,17805.0,0.0,4000,6.683375,NaN,0.000000,2.244375


In [109]:
ults_per_game = data['ults per game'].to_numpy()
winrate = data['Win Rate'].to_numpy()
character_name = data['Name'].to_list()
character_type = data['Type'].to_list()
types = ['Vanguards','Duelists','Strategists']
c_ik_list=[]
for i_idx,i in enumerate(character_name):
        c_ik_list.append((character_name[i_idx],character_type[i_idx]))
c_ik_list
average_ults_per_game = np.mean(ults_per_game)
average_winrate = np.mean(winrate)
average_ults_per_game,average_winrate
A=average_winrate/average_ults_per_game


In [110]:
c_ik_list
len(c_ik_list)

34

Lp implementation

In [ ]:
def one_to_one_best_team_comp():
    A=average_winrate/average_ults_per_game
    B=1/6
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate)



In [ ]:
one_to_one_best_team_comp()

The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Cloak & Dagger', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 49.035000000000004


In [176]:
def highest_winrate_best_team_comp():
    A=1.5
    B=1/6
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate,'%')

In [177]:
highest_winrate_best_team_comp()

The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Rocket Racoon', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 50.30000000000001 %


In [143]:
def best_team_comp():
    A=1
    B=0
    num_types = len(types)
    num_chars = len(character_name)

    A_c_ik=np.ones(num_chars)
    b_ik=[6] #team comp constraint

    A_ck=np.zeros((num_types,num_chars))

    for k_idx,k in enumerate(types):
        for c_idx,c in enumerate(character_name):
            if c_ik_list[c_idx][1] == types[k_idx]:
                A_ck[k_idx][c_idx]=1
    
    b_ck=np.full(num_types,2,dtype=int)

    A_eq = np.vstack((A_c_ik,A_ck))
    b_eq = np.hstack((b_ik,b_ck))
    c=np.ones(num_chars)

    for i_idx,i in enumerate(character_name):
        c[i_idx]=-A*ults_per_game[i_idx]-B*winrate[i_idx]# -min = max
    
    results=linprog(c,A_eq=A_eq,b_eq=b_eq,bounds=(0,1),method='highs')

    team_comp = []
    u_i = []
    x_i= []
    
    for i_idx,i in enumerate(results.x):
        if i>0:
            team_comp.append(c_ik_list[i_idx])
            u_i.append(ults_per_game[i_idx])
            x_i.append(winrate[i_idx])

    total_ults = math.floor(np.sum(u_i))
    avg_winrate = np.mean(x_i)
    
    print('The best team composition is:',team_comp)
    print('The total ults per game is:',total_ults) 
    print('The average winrate is:',avg_winrate)

In [144]:
best_team_comp()

The best team composition is: [('Groot', 'Vanguards'), ('Loki', 'Strategists'), ('Storm', 'Duelists'), ('Venom', 'Vanguards'), ('Cloak & Dagger', 'Strategists'), ('Squirrel Girl', 'Duelists')]
The total ults per game is: 46
The average winrate is: 49.035000000000004
